# Predicting Future Sales by the Teletubbies

In this notebook, we are looking to **predict future sales** for every store using historical data.

----

Mission brief:
*"You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge."*

To make our predictions, we will follow the OSEMN data science framework
1. **O**btain the data 
    * Load the prerequisite packages
    * Load the data
2. **S**crub the data
    * Identify data oddities
    * Identify missing values
3. **E**xplore the data
    * Examine the nature of the variables
    * Explore the time series data
    * Conduct feature engineering
4. **M**odel the data 
    * Create the model
    * Fine-tune the model
5. I**n**terpret the data

## Step \#1: Obtain the data

### Load the prerequisite packages

In [ ]:
#Standard data science packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Data visualisation packages
import matplotlib.pyplot as plt
import seaborn as sns

### Load the data

In [ ]:
#Check what files are in the input directory
import os
print(os.listdir("../input/competitive-data-science-predict-future-sales"))

In [ ]:
#Import the data
raw_sales = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
raw_items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
raw_shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
raw_test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
raw_sample = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
raw_item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")

### Do some initial exploration of the data

#### Sales data

The training dataset contains a row for each item-shop combination by day and provides the items sold per day (item_cnt_day). Some questions to be validated:
- Is this unique by date x shop ID x item ID combination?
- Why are some values for item_cnt_day negative? Are these refunds?

In [ ]:
df_sales  = raw_sales.copy()
df_sales.head()

In [ ]:
df_sales.shape

#### Submission data

The submission format is to provide item_cnt_month for each ID.

To be validated:
* What ID is this?

In [ ]:
df_ssubmission = raw_sample.copy()
df_ssubmission.head()

In [ ]:
df_test = raw_test.copy()
df_test.head()

## Step \#2: Scrub the data 

### Join the reference data to the sales data

#### First join: shop data

We will join the shop data to the sales data. We will perform a left join and we will check that there are no duplicated rows.

In [ ]:
df_shops = raw_shops.copy(deep = True)
print("Dataframe size: ", df_shops.shape)
df_shops.head()

In [ ]:
df_sales2 = pd.merge(left = df_sales, right = df_shops, on = 'shop_id')
print("Pre-join row count:", len(df_sales.index))
print("Post-join row count:", len(df_sales2.index))
if len(df_sales.index) == len(df_sales2.index):
    print("You're safe - no row duplication found!")
else: 
    print("WARNING: Rows have been duplicated!")
df_sales2.head()

#### Next set of joins: item and item categories data

Next, we will add the item names and item categories to the sales data 

In [ ]:
df_items = raw_items.copy(deep = True)
print("Dataframe size: ", df_items.shape)
df_items.head()

In [ ]:
df_item_categories = raw_item_categories.copy(deep = True)
print("Dataframe size: ", df_item_categories.shape)
df_item_categories.head()

In [ ]:
df_items2 = pd.merge(left = df_items, right = df_item_categories, on = 'item_category_id')
print("Pre-join row count:", len(df_items.index))
print("Post-join row count:", len(df_items2.index))
if len(df_items.index) == len(df_items2.index):
    print("You're safe - no row duplication found!")
else: 
    print("WARNING: Rows have been duplicated!")
df_items2.head()

In [ ]:
df_sales3 = pd.merge(left = df_sales2, right = df_items2, on = 'item_id')
print("Pre-join row count:", len(df_sales2.index))
print("Post-join row count:", len(df_sales3.index))
if len(df_sales2.index) == len(df_sales3.index):
    print("You're safe - no row duplication found!")
else: 
    print("WARNING: Rows have been duplicated!")
df_sales3.head()

### Check for oddities in the data
-------

#### Findings
* ~75% of the rows have 1 in the item_cnt_day column
* The minimum value in item_cnt_day column is -2.2
* The highest value in item_cnt_day column is 2,169

In [ ]:
df_sales3.describe()

In [ ]:
boxplot = df_sales3.boxplot(column ='item_cnt_day')

### Check for missing values

---

#### Findings
* There are no missing values in the cells

In [ ]:
#Check if there are any NaNs in the data by converting all sales to Booleans and seeing if any are 1
df_sales3.isnull().values.any()

## Step \#3: Explore the data

### Examine the variables

We should explore the nature of the static data such as the number of products by item category and by store.

#### How many items exist within each item category?

In [ ]:
# Group the items df by the item category id
items_per_cat = df_items2.groupby(['item_category_id']).count()

# Sort the item categories by the number of items in them in descending order
items_per_cat = items_per_cat.sort_values(by = 'item_id', ascending = False)

# Filter only the top 10 item categories
items_per_cat = items_per_cat.iloc[0:20].reset_index()
items_per_cat.head()

In [ ]:
plt.figure()
ax = sns.barplot(data = items_per_cat, x = 'item_category_id', y = 'item_id', color = "mediumblue", order = items_per_cat.item_category_id)
plt.title("The number of items in the top 10 largest item categories")
plt.ylabel("Number of item IDs")
plt.xlabel("Item category ID")
plt.show()

### Explore the time series data

As this is sales data, we should examine how it varies over time and how it varies by product.

# KYY model playground

In [ ]:
kyy_raw_data = df_sales3.copy()
kyy_raw_data['year']=pd.DatetimeIndex(kyy_raw_data['date']).year
kyy_raw_data.head()

In [ ]:
kyy_data_test = kyy_raw_data[['shop_id', 'item_id', 'item_cnt_day', 'item_price', 'year']]
kyy_data_test.head()

In [ ]:
kyy_data_test.dtypes

In [ ]:
kyy_data_test2 =kyy_data_test.astype({"shop_id": 'category' , "item_id": 'category', 'item_cnt_day': int, 'item_price': float, 'year':int})

In [ ]:
X = kyy_data_test2[['shop_id', 'item_id','item_price','year']]
y = kyy_data_test2[['item_cnt_day']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
import seaborn as seabornInstance 
regressor_kyy = LinearRegression()  
regressor_kyy.fit(X_train, y_train)

In [ ]:
coeff_df = pd.DataFrame(regressor_kyy.coef_, X.columns, columns=['Coefficient'])  
coeff_df

In [ ]:
y_pred = regressor_kyy.predict(X_test)

In [ ]:
df_pred_kyy = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_pred_kyy1 = df_pred_kyy.head(25)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))